In [12]:
import csv
from math import sqrt
import os
from copy import deepcopy

In [13]:
notebook_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("lin_regr.ipynb")), os.pardir))
data_folder = "/data/"

In [14]:
airline_delay_causes = []
columns = []
target_variable = "DEP_DELAY"
target_col = []

Считывание данных из файла в array

In [15]:
with open(notebook_path + data_folder + '2017.csv', newline='\n') as csv_file:
	csv_reader = csv.reader(csv_file, delimiter=',')
	line_count = 0
	for row in csv_reader:
		if line_count == 0:
			columns = row.copy()
			line_count += 1
		else:
			airline_delay_causes.append(row)
			line_count += 1

In [16]:
for col in columns:
    print(columns.index(col), " ", col)

0   YEAR
1   MONTH
2   DAY_OF_MONTH
3   DAY_OF_WEEK
4   OP_UNIQUE_CARRIER
5   TAIL_NUM
6   OP_CARRIER_FL_NUM
7   ORIGIN
8   DEST
9   CRS_DEP_TIME
10   DEP_TIME
11   DEP_DELAY
12   TAXI_OUT
13   TAXI_IN
14   CRS_ARR_TIME
15   ARR_TIME
16   ARR_DELAY
17   CANCELLED
18   CANCELLATION_CODE
19   DIVERTED
20   CRS_ELAPSED_TIME
21   ACTUAL_ELAPSED_TIME
22   AIR_TIME
23   FLIGHTS
24   DISTANCE
25   CARRIER_DELAY
26   WEATHER_DELAY
27   NAS_DELAY
28   SECURITY_DELAY
29   LATE_AIRCRAFT_DELAY
30   


Отбор необходимых признаков, отбрасывание столбцов, у которых эти фичи находятся без значения

In [17]:
target_index = 11
# features = [1, 2, 9, 10, 12, 17, 19, 25, 26, 27, 28, 29]
features = [1, 2, 9, 10, 17, 19]		# это фичи, которые можно использовать для предсказания подлета, еще не совершенного
data_train_features = []
data_train_target = []
for row in airline_delay_causes:
	tmp = []
	for col in features:
		if len(row[col]) == 0:
			break
		if len(row[target_index]) == 0:
			break
		else:
			tmp.append(float(row[col]))
	if len(tmp) == len(features):
		data_train_features.append(tmp)
		data_train_target.append(float(row[target_index]))
len(data_train_features)

489170

Функции линейной алгебры

In [18]:
def norm(V):
    ans = 0
    for i in V:
        ans += i**2
    return sqrt(ans)

def sumvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] += a
    return ans

def sumvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] += Y[i]
    return ans

def subvc(X, a):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] -= a
    return ans

def subvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] -= Y[i]
    return ans

def multvc(X, c):
    ans = X.copy()
    for i in range(len(X)):
        ans[i] *= c
    return ans

def multvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = X.copy()
    for i in range(len(Y)):
        ans[i] *= Y[i]
    return ans

def dotvv(X, Y):
    if len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += X[i] * Y[i]
    return ans

def summc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] + c
    return ans

def summm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] + B[i][j]
    return ans

def submc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for row in range(cols_A)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] - c
    return ans

def submm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise Exception("Dim of matrix should be same!")
    rows_A = len(A)
    cols_B = len(B[0])
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            ans[i][j] = A[i][j] - B[i][j]
    return ans

def multmc(A, c):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    ans = [[0 for col in range(cols_A)] for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i][j] = A[i][j] * c
    return ans

def multmv(A, V):
    if len(A) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_V = len(V)
    if cols_A != rows_V:
        raise Exception("Cannot multiply the matrix and vector. Incorrect dimensions.")
    ans = [0 for row in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_A):
            ans[i] += A[i][j] * V[j]
    return ans

def multmm(A, B):
    if len(A) == 0 or len(B) == 0:
        return []
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])
    if cols_A != rows_B:
        raise Exception("Cannot multiply the two matrices. Incorrect dimensions.")
    ans = [[0 for row in range(cols_B)] for col in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                ans[i][j] += A[i][k] * B[k][j]
    return ans

def decvc(V, c):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= c
    return ans

def decvv(V, W):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] /= W[i]
    return ans

def sqrtv(V):
    ans = V.copy()
    for i in range(len(V)):
        ans[i] = sqrt(ans[i])
    return ans

def powv(V, s):
    if s == 0:
        return [1 for el in range(len(V))]
    tmp = powv(V, int(s/2))
    if s % 2 == 0:
        return multvv(tmp, tmp)
    else:
        return multvv(multvv(tmp, tmp), V)

Функции метрик

In [19]:
def MSE(W, X, Y, alpha=None):
    if len(X[0]) != len(W) or len(X) != len(Y):
        raise Exception("Len of vectors should be same!")
    ans = 0
    for i in range(len(X)):
        ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
    return ans

In [20]:
def absv(V):
    ans = V.copy()
    for i in range(len(V)):
        if ans[i] < 0:
            ans[i] = -ans[i]
    return ans

In [21]:
def MSE_l1(W, X, Y, alpha):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ans = 0
	for i in range(len(X)):
		ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
	return ans + (alpha*sum(absv(W)))/len(W)

In [22]:
def MSE_l2(W, X, Y, alpha):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ans = 0
	for i in range(len(X)):
		ans += ((dotvv(W, X[i]) - Y[i])**2)/len(X)
	return ans + (alpha*sum(powv(W, 2)))/len(W)

In [23]:
def R2(W, X, Y):
	if len(X[0]) != len(W) or len(X) != len(Y):
		raise Exception("Len of vectors should be same!")
	ym = sum(Y)/len(Y)
	SSreg = 0
	for i in range(len(X)):
		SSreg += (dotvv(W, X[i]) - ym)**2
	SStot = 0
	for i in range(len(Y)):
		SStot += (Y[i] - ym)**2
	return 1 - (SSreg / SStot)

Функции градиентного спуска

In [24]:
def grad(method, h, W, X, Y, alpha):
	col_X = len(X[0])
	ans = [0 for i in range(col_X)]
	Wn = W.copy()
	for i in range(col_X):
		Wn[i] += h
		ans[i] = (method(Wn, X, Y, alpha) - method(W, X, Y, alpha))/h
		Wn[i] -= h
	return ans

In [25]:
def LeastSquareMethod(method, A, y, alpha, beta=None, gamma=None):
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-8
    h = 1e-8
    L = 0
    Lnew = method(W, A, y, alpha)
    num_steps = 2000
    while abs(L - Lnew) > 1e-6 and num_steps > 0:
        L = Lnew
        W = subvv(W, multvc(grad(method, h, W, A, y, alpha), step))
        Lnew = method(W, A, y, alpha)
        num_steps -= 1
    return W

In [26]:
def SGD(method, A, y, alpha=None, beta=None, gamma=None):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-8
	h = 1e-8
	L = 0
	Lnew = method(W, A, y, alpha)
	num_steps = 50
	while abs(L - Lnew) > 1e-5 and num_steps > 0:
		for i in range(len(A)):
			L = Lnew
			W = subvv(W, multvc(grad(method, h, W, [A[i]], [y[i]], alpha), step))
			Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

In [27]:
def AdaGrad(method, A, y, alpha=None, beta=None, gamma=None):
	col_A = len(A[0])
	W = [1. for row in range(col_A)]
	step = 1e-1
	h = 1e-2
	eps = 1
	L = method(W, A, y, alpha)
	Gnii = [0 for i in range(col_A)]
	temp = grad(method, h, W, A, y, alpha)
	for i in range(col_A):
		Gnii[i] = (temp[i])**2
		W[i] -= step*temp[i]/sqrt(Gnii[i] + eps)
	Lnew = method(W, A, y, alpha)
	num_steps = 2500
	while abs(L - Lnew) > 1e-1 and num_steps > 0:
		L = Lnew
		temp = grad(method, h, W, A, y, alpha)
		for i in range(col_A):
			Gnii[i] += (temp[i])**2
			W[i] -= step*temp[i]/sqrt(Gnii[i] + eps)
		Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

In [28]:
def RMSProp(method, A, y, alpha=None, beta=None, gamma=0.5):
    if gamma is None:
        gamma = 0.5
    col_A = len(A[0])
    W = [1. for row in range(col_A)]
    step = 1e-3
    h = 1e-3
    eps = 1e-3
    L = method(W, A, y, alpha)
    Eg = 0
    tmp = 0
    tmpnew = grad(method, h, W, A, y, alpha)
    Eg = multvc(powv(tmpnew, 2), 1-gamma)
    W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
    Lnew = method(W, A, y, alpha)
    num_steps = 3500
    while abs(L - Lnew) > 1e-2 and num_steps > 0:
        L = Lnew
        tmp = tmpnew
        tmpnew = grad(method, h, W, A, y, alpha)
        Eg = sumvv(multvc(powv(tmp, 2), gamma), multvc(powv(tmpnew, 2), 1-gamma))
        W = subvv(W, decvv(multvc(tmpnew, step), sqrtv(sumvc(Eg, eps))))
        Lnew = method(W, A, y, alpha)
        num_steps -= 1
    return W

In [29]:
def Adam(method, A, y, alpha=None, gamma=0.9, beta=[0.95, 0.95]):
	if gamma is None:
		gamma = 0.9
	if beta is None:
		beta=[0.95, 0.95]
	col_A = len(A[0])
	W =[1. for row in range(col_A)]
	step = 1e-4
	h = 1e-3
	eps = 1e-3
	L = 0
	Lnew = method(W, A, y, alpha)
	tmp = grad(method, h, W, A, y, alpha)
	mt = multvc(tmp, 1-beta[0])
	_mt = decvc(mt, 1-beta[0])
	vt = multvc(powv(tmp, 2), 1-beta[1])
	_vt = decvc(vt, 1-beta[1])
	W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
	L = Lnew
	Lnew = method(W, A, y, alpha)
	num_steps = 4000
	while abs(L - Lnew) > 1e-3 and num_steps > 0:
		L = Lnew
		tmp = grad(method, h, W, A, y, alpha)
		mt = sumvv(multvc(mt, beta[0]), multvc(tmp, 1-beta[0]))
		_mt = decvc(mt, 1-beta[0])
		vt = sumvv(multvc(vt, beta[1]), multvc(powv(tmp, 2), 1-beta[1]))
		_vt = decvc(vt, 1-beta[1])
		W = subvv(W, decvv(multvc(_mt, step), sqrtv(sumvc(_vt, eps))))
		Lnew = method(W, A, y, alpha)
		num_steps -= 1
	return W

Проверка работы написанных методов

In [30]:
alpha = 0.8

In [31]:
W = LeastSquareMethod(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (LSM) = ", W)
print("LSM MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("LSM R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (LSM) =  [0.9953843333362329, 0.9912764970722492, -0.030234118632279205, 0.026670810119412636, 0.9999973371586748, 0.9999944317225982]
LSM MSE (train) =  2002.2457050078795
LSM R2  (train) =  0.9754301989008686
LSM MSE (test) =  3874.707710694763
LSM R2  (test) =  0.9523828784925616


In [32]:
W = LeastSquareMethod(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (LSM) = ", W)
print("LSM MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("LSM MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("LSM R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (LSM) =  [0.9953816957779509, 0.9912738908642496, -0.030235705838322247, 0.02667242335883202, 0.999994669784428, 0.9999917671279945]
LSM MSE (train) =  2002.2425789832391
LSM R2  (train) =  0.9754282678195829
LSM MSE (test) =  3874.723609692847
LSM R2  (test) =  0.952380209826508


In [33]:
W = SGD(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (SGD) = ", W)
print("SGD MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("SGD R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (SGD) =  [0.976745509449176, 0.9568801035587158, -0.04171559135752234, 0.031773751398039396, 0.9999333369902406, 0.9999122878904186]
SGD MSE (train) =  2083.540037125873
SGD R2  (train) =  0.9411089406417071
SGD MSE (test) =  4269.506835207454
SGD R2  (test) =  0.8602281538817897


In [34]:
W = SGD(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (SGD) = ", W)
print("SGD MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("SGD MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("SGD R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (SGD) =  [0.976681191682568, 0.956817777025685, -0.0417173968673723, 0.03177632594750812, 0.9998666762864902, 0.9998456290556169]
SGD MSE (train) =  2083.5288337626425
SGD R2  (train) =  0.9411070938475075
SGD MSE (test) =  4269.5337636149525
SGD R2  (test) =  0.8602219440403742


In [35]:
W = AdaGrad(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (AdaGrad) = ", W)
print("AdaGrad MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("AdaGrad R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (AdaGrad) =  [0.012829392561839447, 0.01288933872790098, 0.00253380175602242, 0.004635717533088014, -0.004999999999930781, -0.011191199387971695]
AdaGrad MSE (train) =  2008.6059753656036
AdaGrad R2  (train) =  0.9922200033464981
AdaGrad MSE (test) =  3773.6001646355617
AdaGrad R2  (test) =  0.9330717935904191


In [36]:
W = AdaGrad(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (AdaGrad) = ", W)
print("AdaGrad MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("AdaGrad MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("AdaGrad R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (AdaGrad) =  [0.012846934300439136, 0.012898564238540884, 0.002533758474932849, 0.004635676121061766, 0.0454594756634639, -0.012611935473927319]
AdaGrad MSE (train) =  2008.6063164073314
AdaGrad R2  (train) =  0.9922200843895486
AdaGrad MSE (test) =  3773.5995550703256
AdaGrad R2  (test) =  0.9330735887075834


In [37]:
W = RMSProp(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.12345490214943651, 0.02291121459752586, -0.04038965481951087, 0.04853582649719525, -1.4266020246148374e-05, -0.0007839453149997187]
RMSProp MSE (train) =  1919.2287152145186
RMSProp R2  (train) =  0.941051961165688
RMSProp MSE (test) =  4147.3847840811295
RMSProp R2  (test) =  0.8765054874327615


In [38]:
W = RMSProp(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.12345490214943651, 0.02291121459752586, -0.04038965481951087, 0.04853582649719525, -1.4266020246148374e-05, -0.0007839453149997187]
RMSProp MSE (train) =  1919.2287152145186
RMSProp R2  (train) =  0.941051961165688
RMSProp MSE (test) =  4147.3847840811295
RMSProp R2  (test) =  0.8765054874327615


In [39]:
W = RMSProp(MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha=0.5)
print("\nW (RMSProp) = ", W)
print("RMSProp MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("RMSProp MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("RMSProp R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (RMSProp) =  [-0.13601346139452763, 0.029329372325142718, -0.040391212164295816, 0.04853953224299829, -0.0004997677978625556, -0.3571148411921018]
RMSProp MSE (train) =  1919.1887049590537
RMSProp R2  (train) =  0.9410425334980366
RMSProp MSE (test) =  4146.81853723521
RMSProp R2  (test) =  0.876670886311853


In [40]:
W = Adam(MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha=0.95, gamma=0.95, beta=[0.95, 0.95])
print("\nW (Adam) = ", W)
print("Adam MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("Adam R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (Adam) =  [-0.0014514810513464906, -0.0007954289511370477, -0.0033788423049536708, 0.009847559972595444, -0.0005251126123069441, -0.0008164709046152803]
Adam MSE (train) =  1988.775701209403
Adam R2  (train) =  0.9927080286383675
Adam MSE (test) =  3841.559863687825
Adam R2  (test) =  0.9213593546271185


In [41]:
W = Adam(MSE_l2,data_train_features[:1000], data_train_target[:1000], alpha, gamma=0.95, beta=[0.9, 0.9])
print("\nW (Adam) = ", W)
print("Adam MSE (train) = ", MSE(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam R2  (train) = ", R2(W, data_train_features[:1000], data_train_target[:1000]))
print("Adam MSE (test) = ", MSE(W, data_train_features[1000:2000], data_train_target[1000:2000]))
print("Adam R2  (test) = ", R2(W, data_train_features[1000:2000], data_train_target[1000:2000]))


W (Adam) =  [-0.04658733330257577, -0.019639268272707077, -0.0410710362486944, 0.048225087733778586, 0.003212863772234631, -0.06572414421194928]
Adam MSE (train) =  1919.2865334050412
Adam R2  (train) =  0.9441976334835918
Adam MSE (test) =  4194.796127046099
Adam R2  (test) =  0.864794574705339


Функция для кросс-валидации и работа с нею

In [42]:
def CrossValidation(methodGD, methodReg, X, Y, alpha=None, beta=None, gamma=None, folds=5):
    Wf = []
    onefold = int(1000/folds)
    for i in range(folds):
        Wf.append(methodGD(methodReg, X[:i*onefold] + X[(i+1)*onefold:], 
                                      Y[:i*onefold] + Y[(i+1)*onefold:], alpha, beta, gamma))
    W = Wf[0].copy()
    for i in range(1, len(Wf)):
        W = sumvv(W, Wf[i])
    W = decvc(W, len(Wf))
    MSE_train = []
    MSE_test = []
    R2_train = []
    R2_test = []
    for i in range(folds):
        MSE_train.append(MSE(W, X[:i*onefold] + X[(i+1)*onefold:], 
                                Y[:i*onefold] + Y[(i+1)*onefold:]))
        MSE_test.append(MSE(W, X[i*onefold:(i+1)*onefold], 
                                Y[i*onefold:(i+1)*onefold]))
        R2_train.append(R2(W, X[:i*onefold] + X[(i+1)*onefold:], 
                              Y[:i*onefold] + Y[(i+1)*onefold:]))
        R2_test.append(R2(W, X[i*onefold:(i+1)*onefold], 
                             Y[i*onefold:(i+1)*onefold]))
    print("\nW " + str(methodGD.__name__) + " (" + str(methodReg.__name__) + ") = ", W)
    print(str(methodGD.__name__) + " MSE (train) = ", sum(MSE_train)/len(MSE_train))
    print(str(methodGD.__name__) + " R2  (train) = ", sum(R2_train)/len(R2_train))
    print(str(methodGD.__name__) + " MSE (test) = ", sum(MSE_test)/len(MSE_test))
    print(str(methodGD.__name__) + " R2  (test) = ", sum(R2_test)/len(R2_test))

In [43]:
CrossValidation(LeastSquareMethod, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W LeastSquareMethod (MSE_l1) =  [0.995383908286658, 0.9912779703536672, -0.030613017555288023, 0.027035730652460187, 0.999997336441993, 0.9999944260818665]
LeastSquareMethod MSE (train) =  2001.594267327019
LeastSquareMethod R2  (train) =  0.9737671627150604
LeastSquareMethod MSE (test) =  2001.5942673270208
LeastSquareMethod R2  (test) =  0.9042755820335765


In [44]:
CrossValidation(LeastSquareMethod, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W LeastSquareMethod (MSE_l2) =  [0.9953812616225605, 0.99127534520062, -0.030614617075889326, 0.027037356308528614, 0.9999946689760236, 0.9999917580243164]
LeastSquareMethod MSE (train) =  2001.5911740475326
LeastSquareMethod R2  (train) =  0.9737651429498871
LeastSquareMethod MSE (test) =  2001.5911740475312
LeastSquareMethod R2  (test) =  0.9042724935887916


In [45]:
CrossValidation(SGD, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha, beta=None, gamma=None)


W SGD (MSE_l1) =  [0.9806418433898759, 0.9642396015495178, -0.045980734772653055, 0.03623279995981854, 0.9999466703660523, 0.9999298905745754]
SGD MSE (train) =  2073.2126668354704
SGD R2  (train) =  0.9348610129283822
SGD MSE (test) =  2073.212666835468
SGD R2  (test) =  0.8926901298765081


In [46]:
CrossValidation(SGD, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)


W SGD (MSE_l2) =  [0.9805901099555848, 0.9641892118229279, -0.045982715414874774, 0.03623539786083185, 0.999893339557579, 0.9998765609789707]
SGD MSE (train) =  2073.2042156774924
SGD R2  (train) =  0.934858061281765
SGD MSE (test) =  2073.2042156774914
SGD R2  (test) =  0.8926860299885053


In [47]:
CrossValidation(AdaGrad, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)


W AdaGrad (MSE_l1) =  [0.012829467196885536, 0.012890228487844005, 0.0025341230249684606, 0.004634270493881597, -0.004999999998482189, -0.01152603439607442]
AdaGrad MSE (train) =  2008.6100650561086
AdaGrad R2  (train) =  0.991613486161538
AdaGrad MSE (test) =  2008.6100650561068
AdaGrad R2  (test) =  0.9534157607850471


In [48]:
CrossValidation(AdaGrad, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)


W AdaGrad (MSE_l2) =  [0.012847009097718071, 0.012899457675121332, 0.002534079742210122, 0.004634229042787848, 0.04545947673677503, -0.01278466622165022]
AdaGrad MSE (train) =  2008.610417748374
AdaGrad R2  (train) =  0.9916135629813952
AdaGrad MSE (test) =  2008.6104177483753
AdaGrad R2  (test) =  0.9534152949951444


In [49]:
CrossValidation(RMSProp, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)


W RMSProp (MSE_l1) =  [-0.1284688444520517, 0.026553389305338525, -0.04320518843998984, 0.05090781074148994, -1.4266020849052841e-05, -0.0009323455022908307]
RMSProp MSE (train) =  1919.0092040100085
RMSProp R2  (train) =  0.9353984826486144
RMSProp MSE (test) =  1919.0092040100092
RMSProp R2  (test) =  0.8651947177342535


In [50]:
CrossValidation(RMSProp, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)


W RMSProp (MSE_l2) =  [-0.13329383207825635, 0.02834872417466955, -0.043203303244974754, 0.05091179303308062, -0.0004999999681632409, -0.22174037927381568]
RMSProp MSE (train) =  1918.979207445915
RMSProp R2  (train) =  0.9353812053271291
RMSProp MSE (test) =  1918.9792074459133
RMSProp R2  (test) =  0.8651590345051391


In [51]:
CrossValidation(Adam, MSE_l1, data_train_features[:1000], data_train_target[:1000], alpha)


W Adam (MSE_l1) =  [-0.015194710012066973, -0.011926254895633119, -0.04130277523941554, 0.04825807441249712, -0.000514904149807688, -0.000729624970722805]
Adam MSE (train) =  1919.7458905334904
Adam R2  (train) =  0.9426984012175043
Adam MSE (test) =  1919.7458905334897
Adam R2  (test) =  0.8817594467060303


In [52]:
CrossValidation(Adam, MSE_l2, data_train_features[:1000], data_train_target[:1000], alpha)


W Adam (MSE_l2) =  [-0.021638483695758333, -0.017726667120012852, -0.04187886296002126, 0.048899452740976936, -0.0005210247401720516, -0.10696604588528]
Adam MSE (train) =  1919.6237011119113
Adam R2  (train) =  0.9412125415700465
Adam MSE (test) =  1919.623701111912
Adam R2  (test) =  0.8794826483013244


Подход к предсказанию вожможности отложения рейса

Найти среднее значение по обучению (delay_yes), которое дает линейная регрессия для таргета, положительного и близкого к нулю.

Если значение на тестовой выборке будет больше или равно delay_yes, то предположить, что вылет будет отложен.

In [53]:
def LinearRegression(W, X, eps):
    return dotvv(W, X) + eps

In [54]:
delay_yes = []
for i in data_train_features[:2000]:
    if data_train_target[data_train_features.index(i)] < 1.0 and data_train_target[data_train_features.index(i)] >= 0.0:
        delay_yes.append(LinearRegression(W, i, 0.0001))
delay_yes = sum(delay_yes)/len(delay_yes)
print(delay_yes)

8.796888239813752


In [55]:
with open(notebook_path + data_folder + 'predict_delay.csv', 'w', newline='\n') as csvfile:
    writ = csv.writer(csvfile, delimiter=',')
    cols = []
    for i in features:
        cols.append(columns[i])
    writ.writerow(cols + ["WILL_BE_DELAYED"])
    for i in data_train_features[2000:]:
        tmp = False
        if LinearRegression(W, i, 0.0001) >= delay_yes:
            tmp = True
        writ.writerow(i + [tmp])